In [90]:
import pandas as pd
import numpy as np

**Dataset**

The ‘TennisAbstract’ website and its associated blog ‘Heavy Topspin’ are resources made available for free by tennis enthusiast Jeff Sackmann. The source is widely referenced in both academic papers (Kovalchik Stephanie, 2016) and in less formal analyses by members of the tennis community and is regularly maintained. The raw data that they link to contains 
information on all ATP tournaments going all the way back to 1968 (start of the open era) in an easily downloadable format and contains the following fields:

In [91]:
''' 
Function to import data from github. Date range must be valid per the given URL
'''

def import_data(start_date: int, end_date: int):
    li = []
    for year in range(start_date, end_date, 1):
        url = f'https://raw.githubusercontent.com/JeffSackmann/tennis_atp/refs/heads/master/atp_matches_{year}.csv'
        data = pd.read_csv(url, header=0)

        li.append(data)

    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame

In [92]:
df = import_data(2010, 2019)
print('Shape:', df.shape)
df.head(5)

Shape: (26591, 49)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2010-339,Brisbane,Hard,32,A,20100103,1,104053,1.0,NaN,...,34.0,29.0,11.0,10.0,3.0,5.0,7.0,4410.0,77.0,598.0
1,2010-339,Brisbane,Hard,32,A,20100103,2,104958,NaN,WC,...,34.0,22.0,14.0,9.0,7.0,10.0,134.0,400.0,78.0,590.0
2,2010-339,Brisbane,Hard,32,A,20100103,3,104755,NaN,NaN,...,58.0,38.0,14.0,14.0,7.0,11.0,52.0,850.0,88.0,568.0
3,2010-339,Brisbane,Hard,32,A,20100103,4,105051,NaN,Q,...,29.0,16.0,15.0,9.0,2.0,5.0,285.0,151.0,28.0,1260.0
4,2010-339,Brisbane,Hard,32,A,20100103,5,104607,4.0,NaN,...,41.0,26.0,14.0,9.0,6.0,9.0,20.0,1655.0,251.0,179.0


In [93]:

'''
Perform data 'cleaning' operations on raw dataframe. List of transformations can be found 
in the precedint markdown cell but in general the goal is to homogenise the dataframe with a 
goal of modelling contests with similar characteristics
'''


def clean_data(df):
    ### Reformat dates using datetime
    df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')
    ### Restrict analysis to hard courts 
    df = df[df['surface'] == 'Hard']
    ### Remove matches with significant missing data (use 'winner aces' here but could use any of missing fields)
    df = df[df['w_ace'].notna()]
    ### Remove abandoned matches - fewer than 24 serves.
    df = df[df['w_svpt'].gt(24) | df['l_svpt'].gt(24)]
    ### Remove matches where 'handedness of one or other players is missing or n/a
    df = df[df['winner_hand'].notna() | df['loser_hand'].notna()]
    #df = df[df['winner_hand'].isnull() | df['loser_hand'].isnull()]


    return df

In [94]:
df = clean_data(df)

In [101]:
df[df.index == 18517]

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
18517,2016-M-DC-2016-G2-AM-M-PUR-ESA-01,Davis Cup G2 R1: PUR vs ESA,Hard,4,D,2016-03-04,1,200619,NaN,NaN,...,67.0,36.0,20.0,14.0,7.0,13.0,NaN,NaN,NaN,NaN


In [96]:
df.shape

(13938, 49)

The following actions were taken to clean the data ahead of analysis: 
•	Dates were standardized using the datetime package. 
•	Matches with empty data fields were removed. 
•	Abandoned matches were removed by deleting games with fewer than 24 service points from winner and loser. 
•	Matches with missing ‘handedness’ were removed 
•	Matches with missing age data were removed 
•	Missing height data was replaced with average height for winner / loser. This is clearly a large assumption but with so many players not having any height data the alternative was to remove them all and it was felt that this would remove too much valuable information from the dataset. 
•	Players with missing age data were removed. 
•	Players without rank data were removed. 
•	Matches with no surface data were removed  
•	Players who had played less than 5 games at ATP level during the study were removed. This was done to address the fact that many of the players on the ATP tour are on the fringes of this level of the game. Places at local tournaments are often secured for local players who are not up to the standard of the other competitors. Players in this category are outliers in the sense that they only manage a handful of games at this level throughout their careers and therefore there is limited data on each of them to have a meaningful impact on a predictive model. The removal of players with fewer than 5 competitive matches at this level over the period of the data is qualitative attempt to 
‘clean’ the data of these contributions. 
 
